In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv('vxx2020.csv')
# housekeeping
df.drop(['symbol'], inplace=True, axis=1)
# add - to date, exdate, and last date after 4 characters
df['date'] = pd.to_datetime(df['date'].astype(str), format='%Y%m%d')
df['exdate'] = pd.to_datetime(df['exdate'].astype(str), format='%Y%m%d')
df['last_date'] = pd.to_datetime(df['last_date'].astype(str), format='%Y%m%d')
df.head()

,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid
0,2020-01-02,2020-01-03,2019-12-17,C,10500,3.90,4.10,0,0,NaN,NaN,NaN,NaN,NaN,131456095
1,2020-01-02,2020-01-03,2019-12-30,C,11000,3.40,3.60,0,0,NaN,NaN,NaN,NaN,NaN,131456096
2,2020-01-02,2020-01-03,2019-12-30,C,11500,3.00,3.05,0,9,2.267680,0.978255,0.030210,0.039436,-16.53450,131456097
3,2020-01-02,2020-01-03,2020-01-02,C,12000,2.52,2.53,30,274,1.908156,0.974564,0.040960,0.045100,-15.88719,131456098
4,2020-01-02,2020-01-03,2020-01-02,C,12500,2.01,2.03,9,390,1.448381,0.977563,0.048564,0.040484,-10.92112,131456099


In [12]:
# add price for VXX
from functions import import_data, check_contango
today = pd.to_datetime('today').strftime('%Y-%m-%d')

tickers = ['^VIX','^VIX3M', 'VXX']

VIX = import_data(tickers[0], '2010-01-01', today, '1d').reset_index()
VIX3M = import_data(tickers[1], '2010-01-01', today, '1d').reset_index()
VXX = import_data(tickers[2], '2020-01-01', today, '1d').reset_index()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
print(df.cp_flag.value_counts())
print('Interesting, there are two more puts than calls total.')

P    203278
C    203276
Name: cp_flag, dtype: int64
Interesting, there are two more puts than calls total.


In [14]:
# drop all rows with 0 volume
print(len(df))
df = df[df['volume'] != 0]
print(len(df))
df.head(20)
print(df.cp_flag.value_counts())


406554
233018
C    137032
P     95986
Name: cp_flag, dtype: int64


After removing all options with zero volume, we have about 30k more active call options than puts.  
Interesting!

In [15]:
# search vxx for Date 2020-01-02
VXX[VXX['Date'] == '2020-01-02'].Close

0    58.040001
Name: Close, dtype: float64

In [16]:
VXX

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-02,58.840000,60.160000,58.000000,58.040001,58.040001,7078100
1,2020-01-03,62.680000,62.959999,59.799999,61.160000,61.160000,12039150
2,2020-01-06,62.959999,63.164001,60.320000,60.320000,60.320000,6514300
3,2020-01-07,60.639999,61.439999,59.400002,60.040001,60.040001,6833775
4,2020-01-08,59.639999,60.000000,57.320000,59.200001,59.200001,12063125
...,...,...,...,...,...,...,...
569,2022-04-05,24.600000,25.889999,24.171000,24.809999,24.809999,4502300
570,2022-04-06,26.010000,26.469999,24.350000,24.379999,24.379999,4747100
571,2022-04-07,24.680000,24.950001,24.010000,24.049999,24.049999,3285200
572,2022-04-08,24.350000,25.000000,23.715000,24.670000,24.670000,2773800


In [20]:
vxx_close = pd.DataFrame()
vxx_close = VXX[["Date", "Close"]]
vxx_close.rename(columns={'Close': 'VXX'}, inplace=True)
df_combined = pd.DataFrame()
df_combined = df.merge(vxx_close, how = 'left', left_on = 'date', right_on = 'Date')
df_combined.head()

C:\Users\S\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,date,exdate,last_date,cp_flag,strike_price,best_bid,best_offer,volume,open_interest,impl_volatility,delta,gamma,vega,theta,optionid,Date,VXX
0,2020-01-02,2020-01-03,2020-01-02,C,12000,2.52,2.53,30,274,1.908156,0.974564,0.040960,0.045100,-15.88719,131456098,2020-01-02,58.040001
1,2020-01-02,2020-01-03,2020-01-02,C,12500,2.01,2.03,9,390,1.448381,0.977563,0.048564,0.040484,-10.92112,131456099,2020-01-02,58.040001
2,2020-01-02,2020-01-03,2020-01-02,C,13000,1.51,1.53,8,389,1.118827,0.971736,0.076274,0.049281,-10.25413,131456100,2020-01-02,58.040001
3,2020-01-02,2020-01-03,2020-01-02,C,13500,1.02,1.04,130,366,0.916016,0.936959,0.178060,0.094039,-15.93107,131456101,2020-01-02,58.040001
4,2020-01-02,2020-01-03,2020-01-02,C,14000,0.53,0.55,833,3756,0.606814,0.873601,0.450219,0.157407,-17.64780,131456102,2020-01-02,58.040001


In [1]:
df

NameError: name 'df' is not defined